## Data Exploration

In [ ]:
import pandas as pd 
import openpyxl as px
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import statsmodels.api as sm
import numpy as np
from scipy.stats import ttest_ind

In [20]:


# Load datasets
tasks_df = pd.read_csv("../original_data/onet_task_statements.csv") 
occ_metadata_df = pd.read_excel("../new_onet_data/excel/occupation_level_metadata.xlsx", engine="openpyxl")
occ_metadata_df.to_csv("../new_onet_data/csv/occupation_level_metadata.csv", index=False)
occ_level_df = pd.read_excel("../new_onet_data/excel/job_zones.xlsx", engine="openpyxl")
occ_level_df.to_csv("../new_onet_data/csv/job_zones.csv", index=False)

# Filter to Incumbent and Occupational Expert tasks and get unique job titles
relevant_tasks = tasks_df[tasks_df['Domain Source'].isin(['Incumbent', 'Occupational Expert'])][['Title', 'Domain Source']].drop_duplicates().reset_index(drop=True)

# Filter only rows about experience breakdown
occ_metadata_exp_incumbant = occ_metadata_df[occ_metadata_df['Item'] == "How Long at Current Job"]
occ_metadata_exp_expert = occ_metadata_df[occ_metadata_df['Item'] == "How Much Experience Performing Work in this Occupation"]

#Pivot it to get a clean format: one row per job title, columns are experience bins
occ_metadata_exp_incumbant_pivot = occ_metadata_exp_incumbant.pivot_table(
    index=['O*NET-SOC Code', 'Title'],
    columns='Response',  # e.g., '1-2 Years', '10+ Years', etc.
    values='Percent',          # Use 'N' for count, or 'Percent' for percentage
    aggfunc='first'      # Use first if it's guaranteed 1 entry per combo
).reset_index()

#Pivot it to get a clean format: one row per job title, columns are experience bins
occ_metadata_exp_expert_pivot = occ_metadata_exp_expert.pivot_table(
    index=['O*NET-SOC Code', 'Title'],
    columns='Response',  # e.g., '1-2 Years', '10+ Years', etc.
    values='Percent',          # Use 'N' for count, or 'Percent' for percentage
    aggfunc='first'      # Use first if it's guaranteed 1 entry per combo
).reset_index()


# Merge domain source and job levels info onto pivoted table
merged_df_incumbant = pd.merge(occ_metadata_exp_incumbant_pivot, relevant_tasks, on='Title', how='left')
merged_df_expert = pd.merge(occ_metadata_exp_expert_pivot, relevant_tasks, on='Title', how='left')

merged_df_incumbant = pd.merge(merged_df_incumbant, occ_level_df, on='Title', how='left')
merged_df_expert = pd.merge(merged_df_expert, occ_level_df, on='Title', how='left')

# Sort by '10+ Years' experience column and save to CSV
sorted_df_incumbant = merged_df_incumbant.sort_values(by='10 Years or More', ascending=False)

sorted_df_expert = merged_df_expert.sort_values(by='10+ Years', ascending=False)


# Mapping of experience bins to estimated average years
experience_weights_experts = {
    "1-2 Years": 1.5,
    "3-4 Years": 3.5,
    "5-9 Years": 7,
    "10+ Years": 17,
    "<1 Year": 0.5,
    "Missing": 0,
    "Never performed work in the occupation": 0
}

experience_weights_incumbants = {
    "1-2 Years": 1.5,
    "3-4 Years": 3.5,
    "5-9 Years": 7,
    "10 Years or More": 17,
    "<1 Year": 0.5,
    "Missing": 0,
    "Never performed work in the occupation": 0
}

# Function to calculate weighted average experience
def compute_weighted_experience_experts(row):
    return sum(row.get(col, 0) * weight/100 for col, weight in experience_weights_experts.items())

def compute_weighted_experience_incumbants(row):
    return sum(row.get(col, 0) * weight/100 for col, weight in experience_weights_incumbants.items())

# Apply to incumbent dataset
sorted_df_incumbant["Avg_Experience_Years"] = sorted_df_incumbant.apply(compute_weighted_experience_incumbants, axis=1)

# Apply to expert dataset
sorted_df_expert["Avg_Experience_Years"] = sorted_df_expert.apply(compute_weighted_experience_experts, axis=1)


sorted_df_incumbant.to_csv("../new_data/domain_source_breakdown_incumbant.csv", index=False)
sorted_df_expert.to_csv("../new_data/domain_source_breakdown_expert.csv", index=False)

In [21]:
# Get average, counts, and percentage breakdown for Job Zone

expert_level_avg = sorted_df_expert["Job Zone"].mean()
incumbant_level_avg = sorted_df_incumbant["Job Zone"].mean()
print(f"Average Job Zone for Expert Tasks: {expert_level_avg}")
print(f"Average Job Zone for Incumbant Tasks: {incumbant_level_avg}")

# For expert tasks
print("Expert Job Zone counts:")
print(sorted_df_expert["Job Zone"].value_counts())

# For incumbent tasks
print("Incumbent Job Zone counts:")
print(sorted_df_incumbant["Job Zone"].value_counts())

# For expert tasks
print("Expert Job Zone percentage breakdown:")
print((sorted_df_expert["Job Zone"].value_counts(normalize=True) * 100).round(2))

# For incumbent tasks
print("Incumbent Job Zone percentage breakdown:")
print((sorted_df_incumbant["Job Zone"].value_counts(normalize=True) * 100).round(2))

# Combine and label source
df_expert = sorted_df_expert[["Job Zone"]].copy()
df_expert["Source"] = "Expert"

df_incumbent = sorted_df_incumbant[["Job Zone"]].copy()
df_incumbent["Source"] = "Incumbent"

combined = pd.concat([df_expert, df_incumbent])

# Total count per Job Zone
total_counts = combined["Job Zone"].value_counts().sort_index()
print("\nTotal Job Zone Counts (Expert + Incumbent):")
print(total_counts)

# Cross-tab: how many in each zone are Expert vs. Incumbent
zone_source_ct = pd.crosstab(combined["Job Zone"], combined["Source"])
print("\nBreakdown of Each Job Zone by Source:")
print(zone_source_ct)

# Percent of each Job Zone that is Expert / Incumbent
zone_source_pct = zone_source_ct.div(zone_source_ct.sum(axis=1), axis=0) * 100
print("\nPercent of Each Job Zone by Source:")
print(zone_source_pct.round(2))

# Average years of experience by Job Zone (overall)
combined_experience = pd.concat([
    sorted_df_expert[["Job Zone", "Avg_Experience_Years"]],
    sorted_df_incumbant[["Job Zone", "Avg_Experience_Years"]]
])
avg_exp_by_zone_total = combined_experience.groupby("Job Zone")["Avg_Experience_Years"].mean().round(2)
print("\nAverage Years of Experience by Job Zone (All Sources):")
print(avg_exp_by_zone_total)

# Average years of experience by Job Zone (Experts only)
avg_exp_by_zone_expert = sorted_df_expert.groupby("Job Zone")["Avg_Experience_Years"].mean().round(2)
print("\nAverage Years of Experience by Job Zone (Expert):")
print(avg_exp_by_zone_expert)

# Average years of experience by Job Zone (Incumbents only)
avg_exp_by_zone_incumbant = sorted_df_incumbant.groupby("Job Zone")["Avg_Experience_Years"].mean().round(2)
print("\nAverage Years of Experience by Job Zone (Incumbent):")
print(avg_exp_by_zone_incumbant)



Average Job Zone for Expert Tasks: 4.01025641025641
Average Job Zone for Incumbant Tasks: 2.9306184012066363
Expert Job Zone counts:
Job Zone
4    93
5    58
3    33
2    10
1     1
Name: count, dtype: int64
Incumbent Job Zone counts:
Job Zone
2    272
3    170
4    100
5     92
1     29
Name: count, dtype: int64
Expert Job Zone percentage breakdown:
Job Zone
4    47.69
5    29.74
3    16.92
2     5.13
1     0.51
Name: proportion, dtype: float64
Incumbent Job Zone percentage breakdown:
Job Zone
2    41.03
3    25.64
4    15.08
5    13.88
1     4.37
Name: proportion, dtype: float64

Total Job Zone Counts (Expert + Incumbent):
Job Zone
1     30
2    282
3    203
4    193
5    150
Name: count, dtype: int64

Breakdown of Each Job Zone by Source:
Source    Expert  Incumbent
Job Zone                   
1              1         29
2             10        272
3             33        170
4             93        100
5             58         92

Percent of Each Job Zone by Source:
Source    Exper

In [ ]:
#Run a t-test to compare Job Zone means and chi squared for their distributions

expert_zones = sorted_df_expert["Job Zone"].dropna()
incumbent_zones = sorted_df_incumbant["Job Zone"].dropna()

t_stat, p_val = ttest_ind(expert_zones, incumbent_zones, equal_var=False)

print(f"T-statistic: {t_stat:.3f}, p-value: {p_val:.5f}")

from scipy.stats import chi2_contingency

# Build contingency table
job_zone_dist = pd.crosstab(index=sorted_df_expert["Job Zone"], columns="Expert")
job_zone_dist["Incumbent"] = sorted_df_incumbant["Job Zone"].value_counts()

# Run test
chi2, p, dof, expected = chi2_contingency(job_zone_dist.fillna(0))

print(f"Chi2: {chi2:.3f}, p-value: {p:.5f}")

T-statistic: 14.372, p-value: 0.00000
Chi2: 163.276, p-value: 0.00000


In [ ]:
df_expert = sorted_df_expert[["Title", "Job Zone"]].copy()
df_expert["Domain_Source"] = "Expert"

df_incumbent = sorted_df_incumbant[["Title", "Job Zone"]].copy()
df_incumbent["Domain_Source"] = "Incumbent"

combined_df = pd.concat([df_expert, df_incumbent], ignore_index=True)
combined_df = combined_df.dropna(subset=["Job Zone"])

# Binary encode
combined_df["Source_Binary"] = (combined_df["Domain_Source"] == "Expert").astype(int)

# Prepare data
X = combined_df[["Job Zone"]]
y = combined_df["Source_Binary"]

# === scikit-learn logistic regression ===
model_sklearn = LogisticRegression()
model_sklearn.fit(X, y)

print(f"scikit-learn Coefficient: {model_sklearn.coef_[0][0]:.4f}")
print(f"scikit-learn Intercept: {model_sklearn.intercept_[0]:.4f}")

# Evaluate
preds = model_sklearn.predict(X)
print("Classification Report (scikit-learn):")
print(classification_report(y, preds))

# === statsmodels logistic regression ===
X_sm = sm.add_constant(X)  # Add intercept term
model_stats = sm.Logit(y, X_sm).fit()
print("\nstatsmodels Summary:")
print(model_stats.summary())


scikit-learn Coefficient: 0.8750
scikit-learn Intercept: -4.2685
Classification Report (scikit-learn):
              precision    recall  f1-score   support

           0       0.81      0.86      0.83       663
           1       0.39      0.30      0.34       195

    accuracy                           0.73       858
   macro avg       0.60      0.58      0.58       858
weighted avg       0.71      0.73      0.72       858

Optimization terminated successfully.
         Current function value: 0.457489
         Iterations 6

statsmodels Summary:
                           Logit Regression Results                           
Dep. Variable:          Source_Binary   No. Observations:                  858
Model:                          Logit   Df Residuals:                      856
Method:                           MLE   Df Model:                            1
Date:                Thu, 31 Jul 2025   Pseudo R-squ.:                  0.1464
Time:                        16:35:02   Log-Likelih